# Modelling TOI-1233 with `chexoplanet`

This tutorial is a good starting point to understand the `chexoplanet` code.

Let's import some modules:

In [ ]:
import sys
sys.path.append("/home/hosborn/home1/python")
from chexoplanet import newfit
import numpy as np
import pandas as pd

#### Initialising the model
To set up a chexoplanet model, you need to give it a name. If possible, use the same name which is found in the CHEOPS database (in this case, TOI1233 without a hyphen).

It is worth including a comment specific to this model run, which will enable you to check between different model runs later on.

You may also decide to specify the RA/Dec of the target here (using `astropy.coordinates.SkyCoord` object), which might help to search later. As may specifying the save file location `save_file_loc`.

In [2]:
mod = newfit.chexo_model("TOI1233", comment="example_fit", save_file_loc="/home/hosborn/home1/data/Cheops_data")

#### Adding a TESS lightcurve

`chexoplanet` was specifically build to interact with data from other exoplanet missions natively, as almost all targets will have for example archival TESS photometry.

The easiest way to access archival data is to use `get_tess` which piggy-backs on the functionality of `MonoTools.lightcurve`. This automatically searches Kepler, K2 and TESS data, downloading all available lightcurves:

In [3]:
mod.get_tess(tic=260647166)

Getting all IDs
ts


However, `MonoTools` is not the easiest to install, and `chexoplanet` also allows you to add lightcurves directly by specifying a time, flux and flux_err arrays to `add_lc`. Flux and errors should be in ppt. It's necessary, especially when there are multiple data sources (e.g. K2 & TESS), to also include the name of the lightcurve - in this case 'tess'.

In either case, `chexoplanet` will initialise lightcurves at the location of `mod.lcs[src]` where src is e.g. 'tess'.

In [4]:
##Commented out as this would duplicate the line above
#tess_lc=pd.read_csv("dummy_lightcurve.csv")
#mod.add_lc(time=tess_lc['time'], flux=(tess_lc['flux']-1.0)*1000, flux_err=tess_lc['flux_err']*1000, src='tess')

#### Initialising Stellar Parameters

Once again, this can be automatically read from the TIC using the data scraped by `MonoTools.lightcurve`. This is automatically done in `get_tess`.

Alternatively stellar parameters can be added manually by giving `[value, neg_err, pos_err]` lists to `init_starpars`.

Depending on which relative error is lowest, the final model can either use Mstar or logg, set by giving any function  `use_logg=True`, `use_Mstar=False` (only one can be set).

In [5]:
mod.init_starpars(Rstar=[0.876, 0.007, 0.007],
                  Mstar=[0.897, 0.042, 0.042],
                  Teff=[5660, 61, 61],
                  logg=[4.49, 0.11, 0.11])

#### Adding CHEOPS observations

The simplest way to add a cheops observation is to use `mod.add_cheops_lc()`, specifying the precise filekey wanted.

If `download=True` then the data will be downloaded from Dace. In order to get private data, make sure you have a DACE log-in, have generated a DACE access token which is in your `.dacerc` file and, bizarrely, make sure you are actively logged into Dace on your browser (even then, this API access step may require a couple of attempts).

You can also specify whether to use the standard "Data Reduction Pipeline" output (`DRP=True`) or whether to perform PSF modelling (using `PIPE=True`).

In the case of PIPE data, which I recommend using, make sure that PIPE is downloaded from github at the most up-to-date version (you may require the `BG-star-fits` branch), that it has been installed via `pip install .` in the PIPE directory, that all the necessary config files have been downloaded (these do not come with the github package and you may need to contact PIPE creator Alexis Brandeker), and that the correct locations are set in `PIPE/pipe/config/conf.json`. Running PIPE may be time-consuming and take ~10mins per filekey.

In [6]:
# # Commented out - you will see why in a second
# mod.add_cheops_lc('CH_PR110045_TG002701_V0200',DRP=False,PIPE=True)

However, the easiest way to do all of this is to use `mod.get_cheops()`. This will perform the above step for all CHEOPS data found on DACE.

In [7]:
mod.get_cheops()

['CH_PR100031_TG024301_V0200' 'CH_PR100031_TG023003_V0200'
 'CH_PR100031_TG022701_V0200' 'CH_PR110045_TG002601_V0200'
 'CH_PR100031_TG022901_V0200' 'CH_PR110045_TG002701_V0200'
 'CH_PR100031_TG023001_V0200' 'CH_PR100031_TG023002_V0200']
['CH_PR100031_TG024301_V0200' 'CH_PR100031_TG023003_V0200'
 'CH_PR100031_TG022701_V0200' 'CH_PR110045_TG002601_V0200'
 'CH_PR100031_TG022901_V0200' 'CH_PR110045_TG002701_V0200'
 'CH_PR100031_TG023001_V0200' 'CH_PR100031_TG023002_V0200']


2023-05-02 11:02:19,235 - INFO - Downloading file on location : /home/hosborn/home1/data/Cheops_data/TOI1233/TOI1233_PR100031_TG024301_V0200_dace_download.tar.gz


 Download : 524 MB

2023-05-02 11:03:29,423 - INFO - File downloaded on location : /home/hosborn/home1/data/Cheops_data/TOI1233/TOI1233_PR100031_TG024301_V0200_dace_download.tar.gz



Download done
Succeeded downloading PR100031_TG024301_V0200 with Dace
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-15-00_RPT_COR_DataReduction_V0200.pdf
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-18-11_SCI_RAW_HkCe-SubArray_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-15-00_SCI_RAW_EventReport_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-18-11_DIAG_MOVIE_Lightcurve-RINF_V0200.mp4
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-16-45_SCI_RAW_FullArray_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-18-11_SCI_COR_Lightcurve-DEFAULT_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-15-00_EXT_PRE_StarCatalogue-OBSID0001117140_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-16-20_SCI_RAW_HkCe-FullArray_V0200.fits
PR100031_TG024301_V0200/CH_PR100031_TG024301_TU2020-05-26T09-18-11_SCI_COR_Lightcurve-OPTIMAL_V0200.fits
PR100

KeyboardInterrupt: 

In [ ]:
from pipe import PipeParam, PipeControl, config
print(config.get_conf_paths())

('/shares/home1/hosborn/data/Cheops_data', '/shares/home1/hosborn/data/PIPE_data')


##### Is there really a planet here? 
We can use `mod.model_comparison_cheops()` will fit two models for each filekey with a transiting planet: one with a transit, and one without. This will then allow us to derive a Bayes Factor and assess whether the transit model is justified.

##### Adding RVs
The model supports radial velocities from multiple instruments. These should be added on a per-instrument basis by giving `mod.add_rvs` arrays of time (`x`), RV in m/s (`y`), RV_err in m/s (`yerr`), and the instrument name (`name`).

`npoly_rv` (default=2) sets the order of polynomial to use to fit background drifts.

`rv_mass_prior` can be `'logK'` (default - log amplitude prior), `'K'` (linear amplitude prior), or `'popMp'` (using the planet radius to compute an expected (log)Mp for the planets given the exoplanet population)

In [ ]:
#Getting PFS RV data from Vizier:
from astroquery.vizier import Vizier
pfs_dat=Vizier(catalog="J/ApJS/256/33/table4").query_constraints(TOI="1233")[0].to_pandas()

#Adding RVs to model:
mod.add_rvs(pfs_dat["JD"]-2457000,pfs_dat["RVel"],pfs_dat["e_RVel"],pd.unique(pfs_dat["Inst"])[0])

['PFS2']


#### Adding planets to a model

There are, as before, two ways to add planets to a model:

1) The precise way (using `mod.add_planet()`) - Here you should specify the planet name and then initial values for the modelling, namely: `tcen` (transit epoch; same units as the data in `mod.lcs`), `tcen_err`, `tdur` (duration in days), `depth` (as a ratio), `period` and `period_err` (in days).

2) The lazy way (using `mod.add_planets_from_toi()`) - this scrapes the planetary info from the TOI catalogue in order to add them directly to the model.

In [ ]:
mod.add_planets_from_toi() #Getting planets bcde
mod.add_planet("f", tcen=1793.2786,tcen_err=0.01,tdur=3.27/24,depth=480,period=29.54115,period_err=1e-4) #Adding planet f

### Initialising the TESS lightcurve

Here we have more choices to make.

The first question is how to detrend the TESS data, for which there are two options:

`fit_gp` or `fit_flat`. In the first case we train a SHO celerite GP on out-of-transit data and then co-fit it to the lightcurve alongside the transit mode. In the latter case, we mask the transits and fit a spline to the lightcurve, subtracting it (NB `flat_knotdist` determines the knot/breakpoint distance of the spline - default:`0.9`, and `mask_distance` gives the distance in transit durations away from the expected transit times to mask - default:`0.55`)

Next we also want to know: how much TESS data do we need to use? Using all TESS points will drastically increase runtime. Here are two options:

`cut_oot` (i.e. Cut out-of-transit points), or `bin_oot` (bin out-of-transit data). The former is a good idea when running `fit_flat`, while the latter works best for `fit_gp` (where correlations are important). These two options are parameterised using `cut_distance` - effectively how many transit durations away from the transits should we cut (default is `3.75`). `bin_size` determines the size in days of binned points (default:`1/48`)

In [ ]:
mod.init_lc(fit_gp=False, fit_flat=True, cut_oot=True, bin_oot=False)

### Initialising the CHEOPS lightcurve

With any CHEOPS lightcurve, decorrelation is typically necessary against hyperparameters such as the centroid, background, & trigonometric functions of the roll angle. The question is typically "what decorrelations are necessary?". `chexoplanet` models individual CHEOPS observations using all available parameters (with both linear and quadratic terms). We can then, like `py_cheops`, use statistics to decide which of the decorrelation parameters are statistically likely. The two ways to do this are to simply take parameters who have a statistically non-zero gradient, or to use the Bayes factor.

Other useful options when initialising CHEOPS data:
* `force_no_dydt` - make sure that time (t) is not used as a decorrelator (for in/egresses this could result in detrending away the transit)
*  `make_detren_params_global` - whether to perform a second statistical analysis is then applied between all visits to determine whether detrending parameters come from the same base distribution and can therefore be shared between visits (defaults to True)
* `force_detrend_pars` - a dictionary of parameter lists forcing the model to use certain detrending parameters, e.g. `{'lin':['time','bg'],'quad':['time']}` 
*  `overwrite` - If the CHEOPS visit has already been modelled, then the default is to reload that model from file and use the derived parameters, so setting `overwrite=True` enforces a new model run.
* `signif_thresh` - the threshold to use in sigma for a "significant" detrending. Defaults to a conservative 1.25
* `linpars` & `quadpars` - the initial set of detrending parameters to start from. If `None`, the following are used for both linear and quadratic: `['sin1phi', 'cos1phi', 'sin2phi', 'cos2phi', 'sin3phi', 'cos3phi', 'bg', 'centroidx', 'centroidy', 'time', 'smear', 'deltaT']`

In [ ]:
mod.init_cheops(use_bayes_fact=True, use_signif=False, overwrite=False)

### Initialising the full model

Now we can have `chexoplanet` build the entire model, using both survey (e.g. TESS) data and multiple CHEOPS visits.

There are some important things to potentially add at this stage:

##### Additional CHEOPS roll angle detrending
CHEOPS data typically contains short-timescale variations as a function of roll angle that are not well absorbed by the trigonometric functions. Hence, GPs or splines as a function of roll angle are typically used. These can be activated here using `fit_phi_gp` (which uses a celerite SHM kernel) or `fit_phi_spline` (default; uses a bspline with a breakpoint cadence of `spline_bkpt_cad=9` degrees). If a spline is used, `spline_order` sets the order (default:3) and `common_phi_model=True` ensures that one spline is used for all CHEOPS visits.

##### TTV fitting
`fit_ttvs=True` can be turned on to fit variable transit timings for each planet. There are three possible priors: `"Normal"` (default), `"Uniform"`, or `"BoundNormal"`. These prior widths are set using `timing_sd_durs` - i.e. the number of transit durations to set as the width of the distributions (default is `0.33`).

##### Limb Darkening
Using `constrain_lds=True` (the default) uses the theoretical Claret limb darkening tables to set a normal prior for TESS/K2/CHEOPS limb darkening parameters. `ld_mult` is used to multiply the theoretical LD uncertainties in order to account for systematic noise (default: `3`). Otherwise, these are allowed to fit using the Kipping reparameterisation. 

##### Circular orbits
By default `assume_circ=True` is set. If a non-circular model is requires, then the eccentricity prior is set with `ecc_prior`. This is typically set to `ecc_prior='auto'` which will pick the kipping distribution for single planet systems and the van eylen prior for multiplanet systems.

In [ ]:
mod.init_model(use_mstar=False, use_logg=True,
               fit_phi_spline=True,fit_phi_gp=False, 
               constrain_lds=True,
               fit_ttvs=False, assume_circ=True)

### Plotting

In [ ]:
mod.plot_phot(src='tess',save=False)
mod.plot_phot(src='k2',save=False)
mod.plot_cheops(save=False,show_detrend=True)
mod.plot_transits_fold(save=False)
mod.plot_rvs(save=False)

### Sampling

In [ ]:
mod.sample_model()

### Saving

In [ ]:
mod.save_timeseries()
mod.save_model_to_file()